#### Импорты

In [1]:
import momepy
import numpy as np
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
import sys
sys.path.append('/Users/polina/Desktop/github/transport_frames')
from transport_frames.src.metrics.indicators import get_intermodal
from transport_frames.src.metrics.grade_territory import grade_territory
from transport_frames.src.graph_builder.graphbuilder import get_graph_from_polygon, get_frame, assign_city_names_to_nodes, convert_list_attr_to_str
from transport_frames.src.metrics.indicators import availability_matrix, find_median
from transport_frames.src.graph_builder.criteria import weight_territory, assign_grades

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Загрузка графа

In [2]:
LOCAL_UTM_CRS = 32636
GLOBAL_CRS = 4326
POLYGON_BUFFER_FOR_ROADS_METERS = 3_000

leningrad_region_poly_osm_relation_id = 'R176095'
saint_petersburg_poly_osm_relation_id = 'R337422'

lo_polygon = ox.geocode_to_gdf(leningrad_region_poly_osm_relation_id, by_osmid=True).to_crs(epsg=LOCAL_UTM_CRS)
spb_polygon = ox.geocode_to_gdf(saint_petersburg_poly_osm_relation_id, by_osmid=True).to_crs(epsg=LOCAL_UTM_CRS).buffer(POLYGON_BUFFER_FOR_ROADS_METERS)
city = lo_polygon.union(spb_polygon).to_crs(epsg=GLOBAL_CRS) #  get lo polygon
russia = ox.geocode_to_gdf("Russia")

citygraph = get_graph_from_polygon(city, crs=LOCAL_UTM_CRS,country_polygon=russia)
citygraph = convert_list_attr_to_str(citygraph)
nx.write_graphml(citygraph, 'spb_roads.graphml')

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 52 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


#### Выделение каркаса

In [3]:
regions = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/regions_of_russia.geojson') #  get regions
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

frame = get_frame(citygraph, regions, city)

/var/folders/ch/1hzm9xv12jg458kt9c3nl6_h0000gn/T/ipykernel_44820/4177023062.py:5: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  frame = get_frame(citygraph, regions, city)
/Users/polina/Desktop/github/transport_frames/transport_frames/src/graph_builder/graphbuilder.py:505: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  n.loc[(n["nodeID"] == path[j]), "weight"] += weight
/Users/polina/Desktop/github/transport_frames/transport_frames/src/graph_builder/graphbuilder.py:506: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  e.loc[


#### Перенос точек на граф

In [4]:
cities = gpd.read_file('/Users/polina/Desktop/github/transport_frames/example_data/18_centers.geojson')
frame = assign_city_names_to_nodes(cities.to_crs(frame.graph['crs']), momepy.nx_to_gdf(frame)[0], frame, name_col='name18', max_distance=1200)

#### Оценка территории

In [5]:
p1 = gpd.read_file('/Users/polina/Desktop/github/transport_frames/example_data/Аэродром лодейнопольское поселение.geojson')
p2 = gpd.read_file('/Users/polina/Desktop/github/transport_frames/example_data/Аэродром лодейнопольское поселение.geojson')
p3 = gpd.read_file('/Users/polina/Desktop/github/transport_frames/example_data/project Светогорского поселения.geojson')
p4 = gpd.read_file('/Users/polina/Desktop/github/transport_frames/example_data/project Шлиссельбург.geojson')
neudobiya = pd.concat([p1,p2,p3,p4])

In [6]:
neudobiya['layer'] = 's'
neudobiya['status'] = 'b'

In [7]:
n_grade = grade_territory(neudobiya, frame, include_priority=True)

#### Расчет критериев оценки состояния территории

In [8]:
polygons188 = gpd.read_file("/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/polygons188.geojson")
points = gpd.read_file("/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/points.geojson")

# p1 = gpd.read_file('../transport_frames/data/geojson/Аэродром лодейнопольское поселение.geojson')
# p2 = gpd.read_file('../transport_frames/data/geojson/Аэродром Сиверск .geojson')
# p3 = gpd.read_file('../transport_frames/data/geojson/project Светогорского поселения.geojson')
# p4 = gpd.read_file('../transport_frames/data/geojson/project Шлиссельбург.geojson')

ferry = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/water_transport_LO.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/airports_local_LO.geojson')
r_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/ЖД остановки.geojson')
fuel = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/fuel.geojson')
b_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/ООПТ ЛО.geojson')

In [9]:
inter = nx.read_graphml('/Users/polina/Desktop/github/transport_frames/preprocessed/ЛО/Ленинградская_область/inter.graphml')

In [11]:
adj_mx = availability_matrix(citygraph, points)
p = find_median(points,adj_mx)

2024-09-01 23:19:32.254 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:84 - Creating adjacency matrix based on provided graph...
2024-09-01 23:19:42.869 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances ...
2024-09-01 23:22:27.081 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:93 - Adjacency matrix done!
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [12]:
p_agg = p[p['to_service'] < np.finfo(np.float64).max].copy()
res = gpd.sjoin(p_agg, polygons188, how='left', op='within').groupby('index_right').median(['to_service']).reset_index()
result_df = pd.merge(polygons188.reset_index(), res, left_on='index', right_on='index_right', how='left').drop(columns=['fid_right']).rename(columns={'to_service': 'in_car'})
result_df = result_df.drop(columns=['index_right']) #in_car - cвязность на машине

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [19]:
result_df

,index,fid,name,layer,2019,2021,2022,2023,2020,geometry,in_car
0,0,7,Самойловское сельское поселение,Бокситогорский муниципальный район,2069,2085,2096,2154,2088,"MULTIPOLYGON (((34.42169 59.68391, 34.42428 59...",3.682300
1,1,2,Большедворское сельское поселение,Бокситогорский муниципальный район,1575,1505,1486,1698,1528,"MULTIPOLYGON (((34.30682 59.71494, 34.30791 59...",3.296896
2,2,6,Пикалевское городское поселение,Бокситогорский муниципальный район,19981,19490,19250,20169,19746,"MULTIPOLYGON (((34.14251 59.48759, 34.13616 59...",3.415549
3,3,3,Борское сельское поселение,Бокситогорский муниципальный район,3312,3301,3272,3393,3279,"MULTIPOLYGON (((34.05997 59.15087, 34.06007 59...",3.503511
4,4,1,Бокситогорское городское поселение,Бокситогорский муниципальный район,15370,15088,14606,15960,15200,"MULTIPOLYGON (((34.06836 59.40315, 34.05535 59...",3.288040
...,...,...,...,...,...,...,...,...,...,...,...
183,183,185,Фёдоровское городское поселение,Тосненский муниципальный район,4460,4450,4562,6083,4536,"MULTIPOLYGON (((30.52049 59.69087, 30.52316 59...",1.844660
184,184,179,Нурминское сельское поселение,Тосненский муниципальный район,3127,3018,2977,3354,3090,"MULTIPOLYGON (((31.01271 59.63362, 31.01394 59...",2.040237
185,185,175,Красноборское городское поселение,Тосненский муниципальный район,5145,4825,4657,4507,5006,"MULTIPOLYGON (((30.58336 59.64993, 30.58367 59...",1.858790
186,186,183,Трубникоборское сельское поселение,Тосненский муниципальный район,1603,1541,1479,1620,1583,"MULTIPOLYGON (((31.13126 59.08942, 31.13522 59...",2.437884


### связность на общ транспорте (аггрегированная по 188)

In [ ]:
from dongraphio import GraphType
adj_mx_inter = availability_matrix(inter,points,graph_type=[GraphType.PUBLIC_TRANSPORT, GraphType.WALK])

In [14]:
p_inter = find_median(points, adj_mx_inter)
points_inter = p_inter[p_inter['to_service'] < np.finfo(np.float64).max].copy()

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [15]:
res_inter = gpd.sjoin(points_inter, polygons188, how="left", predicate="within").groupby('index_right').median(['to_service']).reset_index()
result_df_inter = pd.merge(result_df, res_inter, left_on='index', right_on='index_right', how='left').drop(columns=['index_right', 'fid_right']).rename(columns={'to_service': 'in_inter'})

In [31]:
result_df_inter

,index,fid,name,layer,2019,2021,2022,2023,2020,geometry,in_car,in_inter
0,0,7,Самойловское сельское поселение,Бокситогорский муниципальный район,2069,2085,2096,2154,2088,"MULTIPOLYGON (((34.42169 59.68391, 34.42428 59...",3.682300,6.766013
1,1,2,Большедворское сельское поселение,Бокситогорский муниципальный район,1575,1505,1486,1698,1528,"MULTIPOLYGON (((34.30682 59.71494, 34.30791 59...",3.296896,6.877583
2,2,6,Пикалевское городское поселение,Бокситогорский муниципальный район,19981,19490,19250,20169,19746,"MULTIPOLYGON (((34.14251 59.48759, 34.13616 59...",3.415549,6.106106
3,3,3,Борское сельское поселение,Бокситогорский муниципальный район,3312,3301,3272,3393,3279,"MULTIPOLYGON (((34.05997 59.15087, 34.06007 59...",3.503511,6.385660
4,4,1,Бокситогорское городское поселение,Бокситогорский муниципальный район,15370,15088,14606,15960,15200,"MULTIPOLYGON (((34.06836 59.40315, 34.05535 59...",3.288040,6.051439
...,...,...,...,...,...,...,...,...,...,...,...,...
183,183,185,Фёдоровское городское поселение,Тосненский муниципальный район,4460,4450,4562,6083,4536,"MULTIPOLYGON (((30.52049 59.69087, 30.52316 59...",1.844660,4.120390
184,184,179,Нурминское сельское поселение,Тосненский муниципальный район,3127,3018,2977,3354,3090,"MULTIPOLYGON (((31.01271 59.63362, 31.01394 59...",2.040237,4.664563
185,185,175,Красноборское городское поселение,Тосненский муниципальный район,5145,4825,4657,4507,5006,"MULTIPOLYGON (((30.58336 59.64993, 30.58367 59...",1.858790,3.861638
186,186,183,Трубникоборское сельское поселение,Тосненский муниципальный район,1603,1541,1479,1620,1583,"MULTIPOLYGON (((31.13126 59.08942, 31.13522 59...",2.437884,5.736096


In [16]:
# Grade the territories
graded_gdf = weight_territory(n_grade, r_stops, b_stops, ferry, aero) # name, layer, status, grade, weight

In [17]:
result = assign_grades(graded_gdf[['name', 'geometry', 'grade', 'weight']], result_df_inter[['index', 'fid', 'name', 'geometry', 'in_car', 'in_inter']])
# result.to_file('data/geojson/result_assesment.geojson', driver='GeoJSON')

/Users/polina/Desktop/github/transport_frames/transport_frames/src/graph_builder/criteria.py:167: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  joined.at[idx, "public_transport_grade"] = public_transport_grade


In [23]:
result

,geometry,grade,weight,car_access_quartile,public_access_quartile,car_grade,public_transport_grade,overall_assessment
0,"POLYGON ((278277.965 6784490.204, 279737.375 6...",5,0.35,4,4,2,0.70,1.350
1,"POLYGON ((387861.999 6644712.938, 388202.551 6...",5,1.00,1,1,5,5.00,5.000
2,"MULTIPOLYGON (((529877.020 6730827.593, 529893...",5,0.55,4,3,2,1.65,1.825
